# GenoPhenoCorr Example 
## Phenotypic Traits associate with Variants on Gene MAPK8IP3


### TODO - Intro to package
BLAH BLAH BAahh

In [1]:
import sys
import glob
  
from genophenocorr import *


## Importing Phenopacket Data

The phenopackets were created using [pyphetools](https://github.com/monarch-initiative/pyphetools) with data from [Platzer K., et al. (2019) De Novo Variants in MAPK8IP3 Cause Intellectual Disability with Variable Brain Anomalies](https://pubmed.ncbi.nlm.nih.gov/30612693/) and [Iwasawa S., et al. (2019) De Recurrent de novo MAPK8IP3 variants cause neurological phenotypes](https://pubmed.ncbi.nlm.nih.gov/30945334/). 


In [2]:
allPatients = Cohort('../phenopackets/PlatzerData/*.json')

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


ValueError: Disease ID must include 'OMIM','MONDO', 'ORPHA', or 'DECIPHER'.

You can also add additional phenopackets to the AllPatients class by using AllPatients.add("Path/to/phenopacket.json"). 

In [ ]:
##TO DO - use this to add the additional patient Ben(I think) found
# allPatients.add("json file to last subject")

## Examine the Data


In [ ]:
allPatients.describe_all()

We can also look more in depth as specific attributes of a patient. 

In [ ]:
firstPat = allPatients.all_patients.get('1')
firstPat.describe()

## Run Correlation Analysis
This function runs either a Chi^2^ test or a Fisher Exact test on the counts of each HPO Term that do or don't match a specific criteria. 
Format for function:
RunStats(AllPatients Class, Fun1, Fun2, extraVar_1, extraVar_2)

If Fun1 is CF.is_var_type, then extraVar_1 can be any variant type (missense, nonsense, deletion, etc.) and the function will count by HPO Term all the patients that have that type of variant and do or do not have that HPO Term. This will create the first half of the Contingency Table. Fun2 and extraVar_2 will do the same, creating the second half of the table. 

The example below creates a contingency table: <br>

| Patients with HPO Term and with missense variant | Patients without HPO Term and with missense variant | <br>
| Patients with HPO Term and with no missense variant | Patients without HPO Term and with no missense variant |


In [ ]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')


In [ ]:
for typ in allPatients.all_var_types:
    print(f'Comparing all in {typ} vs out of {typ}')
    compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, typ, typ)
    for typ2 in allPatients.all_var_types:
        if typ2 == typ:
            continue
        print(f'Comparing all in {typ} vs all in {typ2}')
        compare.RunStats(allPatients, CF.is_var_type, CF.is_var_type, typ, typ2)

In [ ]:
for var in allPatients.all_variants:
    print(f'Comparing affecting {var.variant_string} vs all not')
    compare.RunStats(allPatients, CF.is_var_match, CF.is_not_var_match, var, var)

In [ ]:
compare.RunStats(allPatients,CF.in_feature, CF.not_in_feature,'Domain','Domain')

In [ ]:
Protein.findProtein('ENSP00000501096')

In [ ]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')
